# The Quantity Theory of Money Insanity (Part 3)

*QTM as a predictor of inflation*

In the previous section, we looked at QTM as an explanation of hyperinflation. In the six cases of hyperinflation we examined in detail, only one of those cases was consistent with QTM.

In this section, we will look at a weaker form of QTM to see how well it explains inflation in the medium term.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

import qtm

qtm.set_style()

In [ ]:
# If you want to use M3, replace "M1" with "M3" in the next line
data = qtm.oecd.Data("../data/preprocess/", "M1")
data.read()

data_m3 = qtm.oecd.Data("../data/preprocess/", "M3")
data_m3.read()

## More Money, More Problems?

*With apologies to the late, great Notorious B.I.G.*

Scott Sumner [explains QTM](https://www.themoneyillusion.com/money-and-inflation-pt-3-the-quantity-theory-of-money-and-the-great-inflation/) as follows:

> One time changes in the supply of money cause a proportionate rise in the price level in the long run, as compared to where the price level would have been had the money supply not changed.

This is a weaker statement than "inflation is always and everywhere a monetary phenomenon," but, if accurate, it would still be helpful for understanding inflation. We will now proceed to taking a look at that question.

### Harmonizing time series

Recognizing if this statement is reflected in the data is difficult with the time-series plots above, but we can make a visualization where this phenomenon should stand out if true.

Instead of looking at year-over-year changes in percentage terms, we will look at them in terms of ranks. This will make it easier to compare data across countries and investigate general relationships between changes in the supply of money and the price level. 

We first group changes in the money supply and inflation each into quantiles of equal size. We will use 20 quantiles as a compromise between granularity and readability. This gives us a time series of ranks of changes in money supply (`c_m1`) and changes in CPI (`c_cpi`), an excerpt from which is shown below. In this excerpt, covering five years of data from Indonesia, the magnitude of the change in M1 between 1991 and 1992 is in the 5th of 20 quantiles, the change in CPI for the same period was in the 12th of 20 quantiles.

In [ ]:
qdf = qtm.oecd.to_quantile_df(data.annual_df, data.money_col(), "c_cpi", 20)
qdf.loc[["IDN", "JPN"],:].head()

When this is plotted, the movements of the changes in M1 and CPI are easier to follow.

In [ ]:
with mpl.rc_context({'axes.labelsize': 'small', 'axes.titlesize': 'small'}):
    data.annual_ts_fig(sharey=True, df=qdf, ylabel="Change Rank", citex=0.98, citey=0.01)
    fig = plt.gcf()
    fig.suptitle("Inflation & M1 Rank Time Series")
    plt.tight_layout()

### Change ranks (5 years forward)

Since it is acknowledged that changes in money supply need some time to affect inflation, we will use the table above to construct a five forward year series of inflation ranks for each year, categorized by changes in the money supply.

In [ ]:
qtm.oecd.quantile_ts_plot_df(qdf, "c_m1", "c_cpi", 6).loc[["IDN", "JPN"],:].head()

This give us a plot of the trajectory of how inflation ranks evolve over 6 years (the initial year, and 5 subsequent years). If M1 growth is in the top 20% in the first year, we draw the line in red; otherwise, the line is drawn in gray.

In [ ]:
# one country
palette = sns.color_palette()
ap80 = dict(arrowstyle="-|>", facecolor=palette[4], alpha=1, edgecolor=palette[4])
ap20 = dict(arrowstyle="-|>", facecolor=palette[1], alpha=1, edgecolor=palette[1])
fig, ax = plt.subplots()
tdf = qtm.oecd.quantile_ts_plot_df(qdf, "c_m1", "c_cpi", 6).loc[["IDN", "JPN"],:]
tdf = tdf.loc[(slice(None), slice("1992", "1995")), :]
qtm.oecd.quantile_ts_plot(ax, tdf.loc["IDN"], palette[4], palette[1], 
                          15, 0.4, 0.6, None, None, 20, palette[4], 0.3)
ax.annotate("Inflation rank trajectory\nInitial year in bottom 80% M1 growth", 
            (0.1, 12), (0.1, 5.0), arrowprops=ap80, fontsize="xx-small")
ax.annotate("Inflation rank trajectory\nInitial year in top 20% M1 growth", 
            (0.1, 14.1), (0.1, 17.5), arrowprops=ap20, fontsize="xx-small")
ax.annotate("Top 20% range\nin gray", (4, 16.5), fontsize="xx-small")
ax.set_yticks([1, 5, 10, 15, 20])
ax.set_title("Rank Plot");

Applying this idea to all our data gives us this.

In [ ]:
with mpl.rc_context({'axes.labelsize': 'small', 'axes.titlesize': 'small'}):
    data.quantile_ts_fig(0.2, 20, 6)
    fig = plt.gcf()
    fig.suptitle("5Y Inflation Rank Trajectory | Highlight M1")
    plt.tight_layout()

### Change Ranks in Detail

There is a lot to digest here. What we can say is, there are some countries for which Sumner's description of QTM seems to apply quite well, like Japan, and there are others for which it does not apply very well at all, like Switzerland.

In [ ]:
tdf = qtm.oecd.quantile_ts_plot_df(qdf, "c_m1", "c_cpi", 6).loc[["CHE", "JPN"],:]

### Japan

In [ ]:
palette = sns.color_palette()
fig, axs = plt.subplots(2, 1, figsize=(10, 6))

ax = axs[0]
plt.sca(ax)
qtm.oecd.ts_a_plot(ax, qdf.loc["JPN"], slice(None), None, None, 3, data.inflation_color, data.money_color)
ax.axvspan("1959", "1972", color=palette[4], alpha=0.3)
qtm.viz.annot_line(ax, "2002", "High M1 growth,\nlow inflation", ("2001", 12), data.money_color)
ax.set_title("JPN | Rank Time Series")
ax.legend()

ax = axs[1]
plt.sca(ax)
qtm.oecd.quantile_ts_plot(ax, tdf.loc["JPN"], palette[4], palette[1], 
                          15, 0.2, 0.5, None, None, 20, palette[4], 0.3)
ax.set_yticks([1, 5, 10, 15, 20])
ax.set_title("JPN | 5Y Inflation Rank Trajectory")
ax.set_ylabel("Change Rank")
None

For Japan, high M1 growth has historically been a good predictor of high forward inflation. But on closer inspection, we see that most of the high M1 growth years were in the period 1959 &ndash; 1972, the Bretton-Woods era. More recent instances of high M1 growth, like 2002, were **not** followed by high inflation.

### Switzerland

And Switzerland presents a case where high M1 growth has not even historically been a good indicator of high forward inflation.

In [ ]:
palette = sns.color_palette()
fig, axs = plt.subplots(2, 1, figsize=(10, 6))

ax = axs[0]
plt.sca(ax)
qtm.oecd.ts_a_plot(ax, qdf.loc["CHE"], slice(None), None, None, 3, data.inflation_color, data.money_color)
ax.set_title("CHE | Rank Time Series")
ax.legend()

ax = axs[1]
plt.sca(ax)
qtm.oecd.quantile_ts_plot(ax, tdf.loc["CHE"], palette[4], palette[1], 
                          15, 0.2, 0.5, None, None, 20, palette[4], 0.3)
ax.set_yticks([1, 5, 10, 15, 20])
ax.set_title("CHE | 5Y Inflation Rank Trajectory")
ax.set_ylabel("Change Rank")
None

## Change Ranks of Inflation vs. Inflation

Let us make the same type of plot as in the previous section, but instead of using the rank of change in M1, we will use the rank of change in inflation to see how well that predicts future inflation.

In [ ]:
pp_df = qtm.oecd.quantile_ts_plot_df(qdf, "c_cpi", "c_cpi", 6)
with mpl.rc_context({'axes.labelsize': 'small', 'axes.titlesize': 'small'}):
    data.quantile_ts_fig(0.2, 20, 6, pp_df=pp_df)
    fig = plt.gcf()
    fig.suptitle("5Y Inflation Trajectory | Highlight Inflation")
    plt.tight_layout()

For 20 of 28 currencies, if inflation is high in year 1, it has been high for 50% or more of the subsequent 5 years. Current inflation does a better job of predicting the level of future inflation than the change in the money supply does.

We can see focus on this by restricting our view to just those years where inflation is initially in the bottom 80%. If we plot inflation trajectories for these years (top 20% M1 growth in red), it becomes clear that the association between money-supply growth and future inflation is much stronger when inflation is already high. **If inflation is not already high, rapid M1 growth rarely leads to high inflation** in the following five years.

In [ ]:
tdf = qtm.oecd.quantile_ts_plot_df(qdf, "c_m1", "c_cpi", 6)
low_inf_pp_df = tdf[tdf['Year_0'] < 15]
with mpl.rc_context({'axes.labelsize': 'small', 'axes.titlesize': 'small'}):
    data.quantile_ts_fig(0.2, 20, 6, pp_df=low_inf_pp_df)
    fig = plt.gcf()
    fig.suptitle("5Y Inflation Trajectory | Low Initial Inflation | Highlight M1")
    plt.tight_layout()

Beyond that, there is basically no tendency to increased inflation in the years following high M1 growth.

In [ ]:
ndf = pd.concat([pd.to_numeric(low_inf_pp_df[c]) for c in low_inf_pp_df.columns], axis=1)
ndf.subtract(ndf['Year_0'], 0).describe()

In [ ]:
fig, ax = plt.subplots()
tdf = ndf.subtract(ndf['Year_0'], 0)
for idx, r in tdf.iterrows():
    ax.plot(range(len(r)), r, color=data.inflation_color, alpha=0.1)
ax.set_title("Inflation change from first year for high M1 years");

# Conclusions

In his 1970 Wincott Lecture, Milton Friedman said that “inflation is always and everywhere a monetary phenomenon in the sense that it is and can be produced only by a more rapid increase in the quantity.” Maybe this was true given the data he had at his disposal at that time. With the data available now, I think the 2021 version of that statement would be, **“Inflation might in some places and at certain times be a monetary phenomenon. :shrug-emoji:”**

There are theoretical reasons why QTM is plausible, but after looking at OECD data for almost 30 countries over a wide range of years, QTM, at least as it is explained to non-economists, is not borne out in the data. The Equation of Exchange, not QTM, explains the Barro data set; hyperinflation is sometimes, but by no means always, preceded by rapid growth of the money supply; and current inflation predicts the trajectory of inflation over the next 5 years better than the growth in the money supply does. In the real world, high inflation can occur before, after, or independently of increases in the money supply.

As important as the question of what causes inflation is the question of what to do about it. Take the case of Indonesia 1998 again. A regional neighbor devalues their currency, applying pressure on Indonesia to devalue and inflate. Indonesia reacted by giving in, devaluing and increasing the supply of the Rupiah. Would they have experienced better socio-economic outcomes had they not taken these steps? I would be interested to hear answers by monetarists and others.

These questions are not just academic and not just of interest to economists. Every day, new cryptocurrencies are being designed with the intention of replacing cash, and some of these cryptocurrencies have built-in assumptions about inflation and how to limit it. Eventually, cryptocurrencies will be exposed to pressures beyond their control, and a robust monetary design will be imperative to prevent things from going off the rails. 

In the episode of the [Odd Lots podcast on MMT with Stephanie Kelton](https://www.bloomberg.com/news/audio/2021-03-17/how-mmt-won-the-fiscal-policy-debate-podcast), much time is spent discussing inflation and its causes. Around minute 32, Tracy Alloway, asks Kelton to explain her views, and Kelton’s answer is basically, “It’s complicated.” In some ways this is less satisfactory than being able to point to variable X as the cause of inflation, but I think, in its favor, is that “it’s complicated” is a more accurate description of the world.